In [1]:
# Used to track processing time
import time

# Used to write a file with the current datetime
import datetime
import os

from textwrap import dedent

# Used to import LLM and set up mutli-agent crews
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

## Initialize LLM
For our VSCode extension and testing, we're using a popular open-sourced LLM called **Llama3**, the eight-billion parameter version. We download and manage these LLMs using [**Ollama**](https://ollama.com), an open-source platform for running LLMs locally.

Ollama allows us to create and use custom models. For testing purposes, we're creating a Llama3 model with a set seed. A seed means that the same input will produce the same output. (Seeding, in practice, does not produce exactly the same outputs given a certain input; this is due to several reasons, including stochastic sampling, model parameter size, and so on.)

Custom models are stored in the same place, locally, as the default model.

### MacOS
`/Users/[username]/.ollama/models/manifests/registry.ollama.ai/library`

### Windows
`C:\Users\[username]\.ollama\manifests\registry.ollama.ai\library\`

### Linux
`/var/lib/ollama/.ollama/models/manifests/registry.ollama.ai/library`

If you want to initialize a custom model, simply reference the name of the model as you would the default model:

```Python
llm = Ollama(model="[custom_model_name"])
```

This will automatically look in the same directory as the default model.

## Creating a Custom-Seeded Model

For more accurate testing, it's recommended to write a model with a set seed. Luckily, this is very simple.

1. Create a new file
2. Name the file. Do NOT include a file format.
3. Open the file in a text editor and include the following:

---

```bash
# Set the base model
FROM llama3:8b

# Set custom parameters
PARAMETER temperature 0.7  # Optional
PARAMETER seed 42
```
---

4. Save the file.
5. Open a terminal and write the following command: `ollama create [model_name] -f /path/to/my_custom_model`
6. To check if the model was successfully created, run `ollama list`; you should see `[model_name]` in the list

In [2]:
llm = Ollama(model="Llama3")

## Creating Agents

In [3]:
class CrewAgents:

    def variable_name_agent(self):
        return Agent(
            role='Senior Python Developer',
            goal=dedent(
                '''
                To review Python functions and ensure that variable names are
                following best practices. Variable names are concise, descriptive,
                and easy to understand.
                '''
            ),
            backstory=dedent(
                '''
                You're a Python developer who's been programming for decades. You're
                renowned for writing excellent code, and particularly for writing
                high-quality variable names.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def docstring_agent(self):
        return Agent(
            role='Senior Code Documentation Expert',
            goal='Write docstrings for Python functions.',
            backstory=dedent(
                '''
                You're a renowned code documentation expert, specifically in Python.
                You know how to write clean, concise docstrings that include all the
                information relevant to developers trying to understand a function.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )
    
    def quality_assurance_agent(self):
        return Agent(
            role="Senior Python Quality Assurance Tester",
            goal=dedent(
            '''
            To review Python code and ensure that it follows best practices.
            Double-check that variable names are accurate and concise and that
            the docstrings accurately reflect what the function actually does.
            '''
            ),
            backstory=dedent(
            '''
            You're a renowned quality assurance expert. You've been reviewing code
            for decades and ensure that code is flawless, with excellent variable
            names and docstrings.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )
    
    def code_writer_agent(self):
        return Agent(
            role="Senior Code Writer",
            goal=dedent(
            '''
            Take code and strip out everything except the code
            and docstrings.
            '''
            ),
            backstory=dedent(
            '''
            You do an amazing job of getting code and stripping it
            of everything that's not code or documentation.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )

## Create Tasks

In [4]:
class CrewTasks:

    def improve_variable_names(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with variable names that are
                accurate, concise, and easy to understand.
                '''
            ),
            agent=agent
        )

    def add_docstring(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Provide docstrings to the Python code provided below.
                Ensure the documentation is concise, accurate, and easy
                to read.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with a well-written, concise, and
                easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def review_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review code and documentation and ensure it meets the
                highest standards of quality. The variable names are
                clear, concise, and accurate; the docstrings are
                accurate and descriptive, so that any developer reading
                them, even a junior developer, would understand exactly
                what they do and how to use them.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with excellent variable names and
                well-written, concise, and easy to understand docstring.
                '''
            ),
            agent=agent
        )

    def write_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the provided code and remove anything that
                is not code or docstrings. This information will
                then be written to a file that can be immediately used
                within a Python script.

                Python Function
                ---------------
                {provided_code}
                '''
            ),
            expected_output=dedent(
                '''
                Code that consists ONLY of functions and their
                docstrings. All other information is stripped out.
                '''
            ),
            agent=agent
        )

## Read File
Reads a file of functions, each immediately proceeded by a comment.

These functions are read line by line and added to the `functions` list
for further processing.

In [5]:
# Define relative file path
test_suite_file_path = 'test_suite/test_functions_raw_practice.py'

# Save all captured functions to a list
functions = []

with open(test_suite_file_path, 'r', encoding='utf-8') as file:

    function_code      = ''
    capturing_function = False

    for line in file:

        # Check if the line is blank
        if line.strip() == '':

            # If the line is blank and we're capturing
            # a function, we've reached the end of the
            # current function; reset
            if capturing_function:
                functions.append(function_code.strip())
                function_code = ''
                capturing_function = False

        # If line isn't blank, start capturing function
        else:
            function_code += line
            capturing_function = True

    # If the file doesn't end in a blank line,
    # ensure the last function is saved.
    if capturing_function:
        functions.append(function_code.strip())

## Process Functions

In [6]:
# Initialize Agents & Tasks
agents = CrewAgents()
tasks  = CrewTasks()

# Create Agents
variable_name_agent = agents.variable_name_agent()
docstring_agent     = agents.docstring_agent()
qa_agent            = agents.quality_assurance_agent()
code_writer_agent   = agents.code_writer_agent()

# Process Functions
updated_functions = []

for function in functions:

    # Create tasks
    improve_variable_names = tasks.improve_variable_names(variable_name_agent, function)
    add_docstrings         = tasks.add_docstring(docstring_agent, improve_variable_names)
    review_code            = tasks.review_code(qa_agent, add_docstrings)
    write_code_to_file     = tasks.write_code(code_writer_agent, review_code)

    # Create crew & assign tasks
    crew = Crew(
        agents = [
            variable_name_agent,
            docstring_agent,
            qa_agent,
            code_writer_agent
        ],
        tasks = [
            improve_variable_names,
            add_docstrings,
            review_code,
            write_code_to_file
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()

    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">>> Time taken: {end_time - start_time:.2f} seconds <<<\n")

 [DEBUG]: == Working Agent: Senior Python Developer
 [INFO]: == Starting Task: 
            Review the variable names in the Python code. Update
            the variable names so that they're accurate, concise, and
            easy to read and understand. Update the variable names only;
            don't change anything else about the functions, and do not
            add comments anywhere in the code.

            Python Function
            ---------------
            def f(n):
a, b = 0, 1
for _ in range(n):
    a, b = b, a + b
return a



> Entering new CrewAgentExecutor chain...
Thought: I'm excited to dive into this task and make sure those variable names are top-notch!

Final Answer:

def fibonacci_sequence(n):
    current_value, next_value = 0, 1
    for _ in range(n):
        current_value, next_value = next_value, current_value + next_value
    return current_value

Please note that I've updated the variable names to accurately reflect their purpose within the function. The or

2024-05-20 11:36:57,227 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:

```python
def fibonacci_sequence(n: int) -> int:
    """Generate the Fibonacci sequence up to the nth term.

    Parameters:
        n (int): The number of terms in the Fibonacci sequence to generate.

    Returns:
        The nth term of the Fibonacci sequence.

    Example usage: fibonacci_sequence(10) would return the first 10 terms of the Fibonacci sequence.
    """
    initial_value, next_value = 0, 1
    for _ in range(n):
        initial_value, next_value = next_value, initial_value + next_value
    return initial_value
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```python
def fibonacci_sequence(n: int) -> int:
    """Generate the Fibonacci sequence up to the nth term.

    Parameters:
        n (int): The number of terms in the Fibonacci sequence to generate.

    Returns:
        The nth term of the Fibonacci sequence.

    Example usage: fibonacci_sequence(10) would return the first 10 terms of the Fibon

In [11]:

def remove_triple_backticks(func_str):
    """Remove triple backticks from the beginning and end of a string."""
    return func_str.strip('`').strip()

# Check if directory exists; if not, create it
# This directory is relative to the location of this Jupyter Notebook
save_location_file_path = 'test_outputs'  # Removed leading slash for relative path
if not os.path.exists(save_location_file_path):
    os.makedirs(save_location_file_path)

# Create custom datetime output, used for filename
current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H-%M-%S') + '.txt'
output_filename = 'function_output_' + current_datetime
full_file_path = os.path.join(save_location_file_path, output_filename)

# Write to the file
with open(full_file_path, 'w', encoding='utf-8') as f:
    for func in updated_functions:
        cleaned_func = remove_triple_backticks(func)
        f.write(cleaned_func + "\n")

print(f'Functions successfully written to file "{full_file_path}".')

Functions successfully written to file "test_outputs/function_output_24-05-20_11-44-52.txt".
